# Multi agent research team with web search tools and memory

This notebook demonstrates how to build a multi-agent research team using CrewAI. The agents can use tools like web search and memory to collaborate on tasks of Proposing novel LLM research contributions, Evaluating the novelty and impact of proposed contributions and Writing research titles and abstracts. The goal of this system is to streamline the research process by leveraging AI agents for idea generation, evaluation, and writing.


🤖 **Multi agent system as a research assistant** This is a simple multi agent system with:
- Memory to aid agents to remember, reason, and learn from past interactions.
- Web search to for finding a novel and impactful research contribution

By: [Ibrahim Sobh](https://www.linkedin.com/in/ibrahim-sobh-phd-8681757/)

## Install

In [1]:
%pip install -U --quiet  langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00


In [2]:
!pip install --quiet langchain-google-genai

In [3]:
!pip install --quiet crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 30.2 MB/s et

## LLM

In [4]:
from IPython.display import Markdown

In [5]:
# Run this cell and paste the API key in the prompt
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5, top_p=0.85)

In [7]:
from langchain import PromptTemplate

# Chain
prompt = PromptTemplate.from_template("Tell me a very short title for a facebook post about {subject}.")
gen_chain = prompt | llm

In [8]:
# Generate
generated_txt = gen_chain.invoke({"subject": "Software testing"})
generated_txt.content

'Test with Confidence'

## Tools

In [9]:
!pip install --upgrade --quiet  duckduckgo-search  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.0 MB/s eta 0:00:00


In [10]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun(max_results=3)

## Crew


In [11]:
from crewai import Agent, Task, Process, Crew

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [26]:
Researcher = Agent(
    role="AI Researcher",
    goal="Find out a novel LLMs research contribution",
    backstory="""You are a researcher in the area of LLMs. This is crucial for
		finding out a novel and impactful research contribution to the LLMs research.
    You can apply enhacments to the proposed contribution if provided.
		""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=True,  # enable collaboration between agent
    max_rpm = 30,
    llm=llm,
    tools=[search_tool], # web search tool
)

In [18]:
Supervisor = Agent(
    role="AI Supervisor",
    goal="Make evaluation how a proposed contrbution is impactful.",
    backstory="""You are an AI expert supervisor with a deep understanding of AI and LLMs research.
		Your expertise lies not just in knowing the technology but in foreseeing how it can be leveraged to solve real-world problems and drive business innovation.
		Your insights are crucial to enhance proposed contrbutions.""",
    verbose=True,
    allow_delegation=True,  # enable collaboration between agent
    max_rpm = 20,
    llm=llm
)

In [27]:
Writer = Agent(
    role="Technical writer",
    goal="Write a research title and abstract",
    backstory="""You are a technical writer and expert in writing research papers in the area of AI and LLMs.
		Your expertise lies in writing professional title and abstract of a paper given the main idea of the contributions.
		The generated text must be clear and suitable to research papers.""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=False,  # enable collaboration between agent
    max_rpm = 30,
    llm=llm
)

In [28]:
Researcher_task = Task(
    description="""
    Propose a novel and impactful contribution as a new LLMs research or paper proposal.
    """,
    agent=Researcher,
    expected_output='A short paragraph of the new LLMs proposed contribution for a new research paper.'
)

Supervisor_task = Task(
    description="""Evaluate proposed contribution in terms of its impact.
		To have a better proposed contribution, suggest and write a short paragraph with clear only one possible enhacement, if any.
    If no enhacements, then you may suggest to write the title and abstact.
    """,
    agent=Supervisor,
    expected_output='The proposed only one clear enhancment for the proposed contribution. If no enhacement, then you may suggest to write the title and abstact.'
)

Writer_task = Task(
    description="""Analyze and summarize the new research contributions and write a title and abstract of a research paper.
    The title should be professional, intersting, innovative and relevant to the area of AI and LLMs.
    The abstract has to be clear and concise with at least 5 sentences, addressing the mian contribution and expected findings.
		The generated text must be clear and suitable to research papers.
    """,
    agent=Writer,
    expected_output='Title of a paper, and the abstract as a paragraph.'
)

In [29]:
# Define the Crew
crew = Crew(
    agents=[Researcher, Supervisor, Writer],
    tasks=[Researcher_task, Supervisor_task, Writer_task],
    verbose=2,
    process=Process.sequential,
    # memory=True,
    # embedder={
    #     "provider": "google",
    #     "config":{
    #         "model": 'models/embedding-001',
    #         "task_type": "retrieval_document",
    #         "title": "Embeddings for Embedchain"
    #         }
    #     }
)

# Kickoff
result = crew.kickoff()

 [2024-08-11 11:00:00][DEBUG]: == Working Agent: AI Researcher
 [2024-08-11 11:00:00][INFO]: == Starting Task: 
    Propose a novel and impactful contribution as a new LLMs research or paper proposal.
    


> Entering new CrewAgentExecutor chain...
Action: duckduckgo_search
Action Input: {'query': 'What are some novel and impactful contributions to LLMs research?'} 

In this article, we provide a systematic and comprehensive overview of the potentials and limitations of LLMs in clinical practice, medical research and medical education. M-LLMs could provide interpretation of an image, the sentiment of a sound clip, and the context of a video and analyze the structure and content of a full document, in addition to editing text and simulating discussions. In this paper, we focus on health care, medical, and research-related use cases. The CANGARU guidelines are an example of how important steps are being made toward establishing an international consensus in the appropriate use of LLMs a

In [30]:
Markdown(result.raw)

**Title:** A Comprehensive Taxonomy of Large Language Models: Types, Strengths, Weaknesses, and Applications

**Abstract:**

Large language models (LLMs) have emerged as a powerful tool in natural language processing, demonstrating impressive performance in various tasks such as text generation, translation, and question answering. However, the rapidly evolving landscape of LLMs can make it challenging to navigate and understand the different types of LLMs available, their strengths and weaknesses, and their potential applications. This paper presents a comprehensive taxonomy of LLMs, providing a structured and organized overview of the LLM landscape. The taxonomy categorizes LLMs based on their architecture, training data, and capabilities, highlighting their key features and limitations. By providing a comprehensive understanding of the different types of LLMs, this taxonomy aims to facilitate research and development in the field of LLMs and guide practitioners in selecting the most appropriate LLM for their specific applications.